# RuGPT3 Fine-Tuning for Russian Jokes Generation

In [ ]:
!pip install transformers==3.0.2 torch -q

     |████████████████████████████████| 769 kB 7.6 MB/s 
     |████████████████████████████████| 1.3 MB 41.1 MB/s 
     |████████████████████████████████| 3.0 MB 48.3 MB/s 
     |████████████████████████████████| 880 kB 56.6 MB/s 


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 686 (delta 189), reused 165 (delta 165), pack-reused 442
Receiving objects: 100% (686/686), 377.90 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [ ]:
!python data_preparation.py

No. of training examples: 5578 

No. of testing examples: 1395

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/ \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 7 \
    --block_size 1024 \
    --overwrite_output_dir

Downloading: 100% 608/608 [00:00<00:00, 371kB/s]
Downloading: 100% 1.71M/1.71M [00:00<00:00, 4.44MB/s]
Downloading: 100% 1.27M/1.27M [00:00<00:00, 3.37MB/s]
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Downloading: 100% 551M/551M [00:09<00:00, 56.6MB/s]
Epoch:   0% 0/7 [00:00<?, ?it/s]
Iteration:   0% 0/376 [00:00<?, ?it/s]
Iteration:   0% 1/376 [00:02<18:32,  2.97s/it]
Iteration:   1% 2/376 [00:03<08:56,  1.43s/it]
Iteration:   1% 3/376 [00:03<05:53,  1.05it/s]
Iteration:   1% 4/376 [00:04<04:29,  1.38it/s]
Iteration:   1% 5/376 [00:04<03:41,  1.67it/s]
Iteration:   2% 6/376 [00:04<03:12,  1.92it/s]
Iteration:   2% 7/376 [00:05<02:54,  2.11it/s]
Iteration:   2% 8/376 [00:05<

### **Generation**

In [ ]:
import re
import numpy as np
import torch

In [ ]:
np.random.seed(21)
torch.manual_seed(21)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('models')
model = GPT2LMHeadModel.from_pretrained('models')
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
text = '<|startoftextpro-rabotu|>Вышла Вита на стажировку'
inpt = tokenizer.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(),  num_return_sequences=3, max_length=80, repetition_penalty=6.0, do_sample=True, top_k=5, top_p=0.95, temperature=1, pad_token_id=50256)

In [ ]:
re.sub(r'<\|startoftext[\w-]+\|>', '', tokenizer.decode(out[0])).split('\n')[0]

'Вышла Вита на стажировку. И что же? Она говорит:- Я хочу, чтобы меня уважали и ценили.- Почему?- Чтобы платили хорошо!'